In [1]:
import statsmodels.api as sm
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import zscore
%matplotlib widget

In [23]:
# Load the Excel file into a DataFrame
NrSubtype='All' #'L1' or 'All'
analysisfile='SWRPFC_ABdetection_GrandGlobalAB'
directory="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording/Analysis_SWR/"
combined_df = pd.read_excel(f'{directory}{NrSubtype}_{analysisfile}.xlsx', index_col=0)            

analysisfile2='SWRPFC_ABdetection_GrandAverageAB'
directory="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording/Analysis_SWR/"
excel_file = f'{directory}{NrSubtype}_{analysisfile2}.xlsx'
excel_data = pd.read_excel(excel_file, sheet_name=None, index_col=0, header=None) 
dict_SWR_average = {}
for sheet_name, sheet_data in excel_data.items():
    dict_SWR_average[sheet_name] = sheet_data

analysisfile3='SWRPFC_ABdetection_SpikeGrandAverageAB'
directory="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording/Analysis_SWR/"
excel_file = f'{directory}{NrSubtype}_{analysisfile3}.xlsx'
excel_data = pd.read_excel(excel_file, sheet_name=None, index_col=0, header=None) 
dict_SWR_Spikeaverage = {}
for sheet_name, sheet_data in excel_data.items():
    dict_SWR_Spikeaverage[sheet_name] = sheet_data

In [24]:
def clean_string(s):
    return s.replace("[", "").replace("]", "").replace("'", "")

# Apply the function to the specific column
combined_df['SWRStatut'] = combined_df['SWRStatut'].apply(clean_string)

In [25]:
combined_df

,Mice,Session,Session_Time,Unique_Unit,UnitNumber,UnitValue,SWRStatut,SWRNumber,SWRDuration (ms),SWR inside Spdl,...,CalciumActivityAfter,AUC_calciumBefore,AUC_calciumAfter,SpikeActivityPreference,SpikeActivityBefore,SpikeActivityAfter,AUC_spikeBefore,AUC_spikeAfter,Unit_ID,SWR_ID
0,Purple,session11,NaN,4,0,0,UnCoupled,1,42.0,False,...,1.786967,16.055433,16.082567,Before,0.012952,0.0,0.129521,0.0,Purple4,Purplesession111
1,Purple,session11,NaN,4,0,0,UnCoupled,2,36.0,False,...,1.754958,16.360936,15.794485,Before,0.012952,0.0,0.079673,0.0,Purple4,Purplesession112
2,Purple,session11,NaN,4,0,0,UnCoupled,3,40.0,False,...,2.892278,26.988487,26.030274,NaN,0.000000,0.0,0.000000,0.0,Purple4,Purplesession113
3,Purple,session11,NaN,4,0,0,UnCoupled,4,122.0,False,...,2.789589,26.030274,25.106081,NaN,0.000000,0.0,0.000000,0.0,Purple4,Purplesession114
4,Purple,session11,NaN,4,0,0,UnCoupled,5,63.0,False,...,2.333205,21.771652,20.998660,NaN,0.000000,0.0,0.000000,0.0,Purple4,Purplesession115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41956,ThreeColDotsOK,session33,NaN,183,8,84,UnCoupled,107,54.0,False,...,0.000727,0.011592,0.010178,NaN,0.000000,0.0,0.000000,0.0,ThreeColDotsOK183,ThreeColDotsOKsession33107
41957,ThreeColDotsOK,session33,NaN,183,8,84,UnCoupled,108,54.0,False,...,0.000117,0.001859,0.001632,NaN,0.000000,0.0,0.000000,0.0,ThreeColDotsOK183,ThreeColDotsOKsession33108
41958,ThreeColDotsOK,session33,NaN,183,8,84,UnCoupled,109,40.0,False,...,0.177814,2.835093,2.489191,NaN,0.000000,0.0,0.000000,0.0,ThreeColDotsOK183,ThreeColDotsOKsession33109
41959,ThreeColDotsOK,session33,NaN,183,8,84,UnCoupled,110,44.0,False,...,0.232740,3.710842,3.258092,NaN,0.000000,0.0,0.000000,0.0,ThreeColDotsOK183,ThreeColDotsOKsession33110


AUC calcium (here no need to normalized AUC cause episode lenght is the same for everybody)

In [26]:
SWRdurmean = combined_df.groupby('SWR_ID')['SWRDuration (ms)'].mean()
filenameOut = f'{directory}{NrSubtype}_SWRdurations_AB.xlsx'
writer = pd.ExcelWriter(filenameOut)
SWRdurmean.to_excel(writer)
writer.close()
np.mean(SWRdurmean)


In [32]:
AUC_calciumBefore_perUnit = combined_df.groupby('Unit_ID')['AUC_calciumBefore'].mean()
AUC_calciumBefore_perUnit

Unit_ID
Purple103            0.638988
Purple107            0.217596
Purple113            0.635444
Purple119            4.824107
Purple12             3.886037
                       ...   
ThreeColDotsOK88     5.113588
ThreeColDotsOK90     2.969481
ThreeColDotsOK91     6.129810
ThreeColDotsOK936    3.497846
ThreeColDotsOK98     0.348459
Name: AUC_calciumBefore, Length: 242, dtype: float64

In [49]:
# Create df for All SWR
AUC_calciumBefore_perUnit = combined_df.groupby('Unit_ID')['AUC_calciumBefore'].mean()
AUC_calciumAfter_perUnit = combined_df.groupby('Unit_ID')['AUC_calciumAfter'].mean()
allAUCresult = pd.concat([AUC_calciumBefore_perUnit,AUC_calciumAfter_perUnit], axis=1)

# Create dfs for each SWR Statut
SWRStatut_AUC_calciumBefore_perUnit = combined_df.pivot_table(index='Unit_ID', columns='SWRStatut', values='AUC_calciumBefore', aggfunc='mean')
SWRStatutNames=SWRStatut_AUC_calciumBefore_perUnit.columns.tolist()
new_columns = [f'Before_{col}' for col in SWRStatut_AUC_calciumBefore_perUnit.columns]
SWRStatut_AUC_calciumBefore_perUnit.columns = new_columns
SWRStatut_AUC_calciumAfter_perUnit = combined_df.pivot_table(index='Unit_ID', columns='SWRStatut', values='AUC_calciumAfter', aggfunc='mean')
new_columns = [f'After_{col}' for col in SWRStatut_AUC_calciumAfter_perUnit.columns]
SWRStatut_AUC_calciumAfter_perUnit.columns = new_columns
# Combine each SWR Statut df into one
AUC_interleaved_cols = [col for pair in zip(SWRStatut_AUC_calciumBefore_perUnit.columns, SWRStatut_AUC_calciumAfter_perUnit.columns) for col in pair]
AUCresult = pd.concat([SWRStatut_AUC_calciumBefore_perUnit[col] if col in SWRStatut_AUC_calciumBefore_perUnit.columns else SWRStatut_AUC_calciumAfter_perUnit[col] for col in AUC_interleaved_cols], axis=1)

# Create a dict containing each SWR Statut and All SWR
AUCdata_dict = {}
c=0
for i in range(len(SWRStatutNames)):
    columns = AUCresult.columns[c:c+2]
    c=c+2
    AUCdata_dict[SWRStatutNames[i]] = AUCresult[columns].values.tolist()
AUCdata_dict['AllSWR']=allAUCresult.values.tolist()


AUCdata_dict_ActiveOnly={}
for key in AUCdata_dict:
    # Filter out rows with non-zero values
    filtered_rows = [row for row in AUCdata_dict[key] if any(val != 0 for val in row) and np.any(~np.isnan(row))]
    # Update the key with filtered rows
    AUCdata_dict_ActiveOnly[key] = filtered_rows


filenameOut = f'{directory}{NrSubtype}_SWRPFC_AUC_perUnitAB.xlsx'
with pd.ExcelWriter(filenameOut) as writer:
    # Iterate over each key-value pair in the dictionary
    for sheet_name, data_list in AUCdata_dict.items():
        # Convert the list to a DataFrame
        df = pd.DataFrame(data_list, columns=['Before', 'After'])
        # Write the DataFrame to a separate sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)

filenameOut = f'{directory}{NrSubtype}_SWRPFC_AUC_perUnit_Active OnlyAB.xlsx'
with pd.ExcelWriter(filenameOut) as writer:
    # Iterate over each key-value pair in the dictionary
    for sheet_name, data_list in AUCdata_dict_ActiveOnly.items():
        # Convert the list to a DataFrame
        df = pd.DataFrame(data_list, columns=['Before', 'After'])
        # Write the DataFrame to a separate sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [50]:
SpikeActivityPreference_perUnits = combined_df.pivot_table(index='Unit_ID', columns='SpikeActivityPreference', aggfunc='size', fill_value=0)
SpikeActivityPreference_perUnits['%PrefBefore']=(SpikeActivityPreference_perUnits['Before'])/(SpikeActivityPreference_perUnits['Before']+SpikeActivityPreference_perUnits['After'])*100
SpikeActivityPreference_perUnits['%PrefAfter']=(SpikeActivityPreference_perUnits['After'])/(SpikeActivityPreference_perUnits['Before']+SpikeActivityPreference_perUnits['After'])*100
SpikeActivityPreference_perUnits

SpikeActivityPreference,After,Before,%PrefBefore,%PrefAfter
Unit_ID,,,,
Purple103,1,1,50.000000,50.000000
Purple113,1,2,66.666667,33.333333
Purple119,2,1,33.333333,66.666667
Purple12,4,5,55.555556,44.444444
Purple126,0,1,100.000000,0.000000
...,...,...,...,...
ThreeColDotsOK88,17,17,50.000000,50.000000
ThreeColDotsOK90,8,7,46.666667,53.333333
ThreeColDotsOK91,2,4,66.666667,33.333333


In [51]:
CalciumActivityPreference_perUnits = combined_df.pivot_table(index='Unit_ID', columns='CalciumActivityPreference', aggfunc='size', fill_value=0)
CalciumActivityPreference_perUnits['%PrefBefore']=(CalciumActivityPreference_perUnits['Before'])/(CalciumActivityPreference_perUnits['Before']+CalciumActivityPreference_perUnits['After'])*100
CalciumActivityPreference_perUnits['%PrefAfter']=(CalciumActivityPreference_perUnits['After'])/(CalciumActivityPreference_perUnits['Before']+CalciumActivityPreference_perUnits['After'])*100
CalciumActivityPreference_perUnits

CalciumActivityPreference,After,Before,%PrefBefore,%PrefAfter
Unit_ID,,,,
Purple103,2,85,97.701149,2.298851
Purple107,0,22,100.000000,0.000000
Purple113,3,6,66.666667,33.333333
Purple119,2,31,93.939394,6.060606
Purple12,7,61,89.705882,10.294118
...,...,...,...,...
ThreeColDotsOK88,28,418,93.721973,6.278027
ThreeColDotsOK90,13,292,95.737705,4.262295
ThreeColDotsOK91,4,63,94.029851,5.970149


Spike Activity 

In [52]:
# Create df for All SWR
SpikeActivityBefore_perUnit = combined_df.groupby('Unit_ID')['SpikeActivityBefore'].mean()
SpikeActivityAfter_perUnit = combined_df.groupby('Unit_ID')['SpikeActivityAfter'].mean()
allSPresult = pd.concat([SpikeActivityBefore_perUnit,SpikeActivityAfter_perUnit], axis=1)

# Create dfs for each SWR Statut
SWRStatut_SpikeActivityBefore_perUnit = combined_df.pivot_table(index='Unit_ID', columns='SWRStatut', values='SpikeActivityBefore', aggfunc='mean')
SWRStatutNames=SWRStatut_SpikeActivityBefore_perUnit.columns.tolist()
new_columns = [f'Before_{col}' for col in SWRStatut_SpikeActivityBefore_perUnit.columns]
SWRStatut_SpikeActivityBefore_perUnit.columns = new_columns
SWRStatut_SpikeActivityAfter_perUnit = combined_df.pivot_table(index='Unit_ID', columns='SWRStatut', values='SpikeActivityAfter', aggfunc='mean')
new_columns = [f'After_{col}' for col in SWRStatut_SpikeActivityAfter_perUnit.columns]
SWRStatut_SpikeActivityAfter_perUnit.columns = new_columns
# Combine each SWR Statut df into one
SP_interleaved_cols = [col for pair in zip(SWRStatut_SpikeActivityBefore_perUnit.columns, SWRStatut_SpikeActivityAfter_perUnit.columns) for col in pair]
SPresult = pd.concat([SWRStatut_SpikeActivityBefore_perUnit[col] if col in SWRStatut_SpikeActivityBefore_perUnit.columns else SWRStatut_SpikeActivityAfter_perUnit[col] for col in SP_interleaved_cols], axis=1)

# Create a dict containing each SWR Statut and All SWR
SPdata_dict = {}
c=0
for i in range(len(SWRStatutNames)):
    columns = SPresult.columns[c:c+2]
    c=c+2
    SPdata_dict[SWRStatutNames[i]] = SPresult[columns].values.tolist()
SPdata_dict['AllSWR']=allSPresult.values.tolist()


SPdata_dict_ActiveOnly={}
for key in SPdata_dict:
    # Filter out rows with non-zero values
    filtered_rows = [row for row in SPdata_dict[key] if any(val != 0 for val in row) and np.any(~np.isnan(row))]
    # Update the key with filtered rows
    SPdata_dict_ActiveOnly[key] = filtered_rows


filenameOut = f'{directory}{NrSubtype}_SWRPFC_SpikeAct_perUnitAB.xlsx'
with pd.ExcelWriter(filenameOut) as writer:
    # Iterate over each key-value pair in the dictionary
    for sheet_name, data_list in SPdata_dict.items():
        # Convert the list to a DataFrame
        df = pd.DataFrame(data_list, columns=['Before', 'After'])
        # Write the DataFrame to a separate sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)

filenameOut = f'{directory}{NrSubtype}_SWRPFC_SpikeAct_perUnit_Active OnlyAB.xlsx'
with pd.ExcelWriter(filenameOut) as writer:
    # Iterate over each key-value pair in the dictionary
    for sheet_name, data_list in SPdata_dict_ActiveOnly.items():
        # Convert the list to a DataFrame
        df = pd.DataFrame(data_list, columns=['Before', 'After'])
        # Write the DataFrame to a separate sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)
